# Notebook Contents

In this notebook the idea os to generate the necessary enviornment lattices for the multi-agent system which are based on calculation of Eucledian Distances.
Hence the lattices which are generated in this notebook include.
1. Distance form Facades (NSEW)
2. Distance from Roof / FLoor 
3. Quiteness lattice (Distance from source of sound) 

The steps followed in the notebook inculde :
1. Load all the assets required for simulation
2. Load the excel files showing the points of interest (In Quiteness Lattice it is the co-ordinates of the Sound source) 
3. Calculate the distances using Scipy https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html
4. Vizualise the lattices on the building
5. Save the lattices as pickle objects


# Initilization

In [1]:
import os
import sys
import topogenesis as tg
import pickle
import pyvista as pv
import pandas as pd
import trimesh as tm
import scipy as sc
from scipy.spatial import distance
import itertools
from itertools import cycle 
import numpy as np
import numpy.ma as ma
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
from ladybug.sunpath import Sunpath

In [2]:
#Design Details
Self_development_plots_path = os.path.relpath('Site_self_development_plots.obj')
Self_development_backyards_path = os.path.relpath('Site_self_development_backyards.obj')
Site_buildings_path = os.path.relpath('Site_buildings.obj')
Site_green_areas_path = os.path.relpath('Site_green_areas.obj')
Site_roads_path = os.path.relpath('Site_base_roads.obj')
Site_context_shading_path= os.path.relpath('Site_surrounding_buildings_for_shading.obj') 


# Site details
Site_base_path = os.path.relpath('Site_base_block.obj')
Site_surrounding_buildings_path = os.path.relpath('Site_surrounding_buildings.obj')
Site_water_bodies_path = os.path.relpath('Site_water_bodies.obj')
Site_roads_path = os.path.relpath('Site_roads.obj')
Site_other_buildings_path = os.path.relpath('Site_other_buildings.obj')

# load the mesh from file
# Design elements
Self_development_plots_mesh = tm.load(Self_development_plots_path)
Self_development_backyards_mesh = tm.load(Self_development_backyards_path)
Site_building_mesh = tm.load(Site_buildings_path)
Site_green_areas_mesh = tm.load(Site_green_areas_path)
Site_roads_mesh = tm.load(Site_roads_path)
Site_context_shading_mesh = tm.load(Site_context_shading_path)

#Site elements
Site_base_mesh = tm.load(Site_base_path)
Site_surrounding_buildings_mesh = tm.load(Site_surrounding_buildings_path)
Site_water_bodies_mesh = tm.load(Site_water_bodies_path)
Site_roads_mesh = tm.load(Site_roads_path)
Site_other_buildings_mesh = tm.load(Site_other_buildings_path)


# Check if the mesh is watertight
#print(envelope_mesh.is_watertight)
#print(context_mesh.is_watertight)

faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!


# Define Points for Quiteness

## import from Rhino 

In [3]:
# Csv to points 
Complete_file = pd.read_excel('Quiteness_points.xlsx', sheet_name=0,engine='openpyxl',header = None )
design_criterias= list(Complete_file.head(n=0))
array_excel = Complete_file.to_numpy()

In [4]:
Noise_sources = array_excel.T

# Define Points for Closeness Facades

In [5]:
len(Noise_sources)

21

# Vizualise the meshes and points

In [6]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
# Design meshes
p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#f4acb7')
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd',opacity= 0.5)

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_surrounding_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
#p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')

p.add_points( Noise_sources, color='#e63946')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1563.6562028391934, 1809.440031574545, 1616.2765183909512),
 (-142.0987091064453, 103.68511962890625, -89.4783935546875),
 (0.0, 0.0, 1.0)]

# Import lattice

In [7]:
# loading the lattice from csv
lattice_path = os.path.relpath('voxelized_envelope_6m_voxel_size.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

# Visualize the Context Mesh + Envelope Lattice

In [8]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes

# Design meshes
p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_surrounding_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1564.3134508410321, 1810.0972795763837, 1618.43376639279),
 (-142.0987091064453, 103.68511962890625, -87.9783935546875),
 (0.0, 0.0, 1.0)]

# Find the voxel Eucledian coordinates

In [9]:
# convert to lattice
init_lattice = envelope_lattice +1
availability_lattice_voxels = tg.to_lattice(init_lattice, init_lattice)
voxel_coordinates= availability_lattice_voxels.centroids


In [10]:
flattened_lattice = envelope_lattice.flatten()

# Find distances for Quiteness 

The distance from each source of sound is calculated and the minimum distance from them is choosen

In [11]:
Eucledian_distance = sc.spatial.distance.cdist(Noise_sources,voxel_coordinates)

In [12]:
noise_from_each_source = Eucledian_distance.T

In [13]:
Average_quiteness_indexing= np.argmin(noise_from_each_source, axis=1)

In [14]:
Average_quiteness_values = []
for branch,index in zip(noise_from_each_source,Average_quiteness_indexing):
    Average_quiteness_values.append(branch[index])
    

In [15]:
Quiteeness_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Average_quiteness_values))])

In [16]:
padded_array = np.array(Quiteeness_lattice_padded)

In [17]:
Quiteness_lattice_np = Quiteeness_lattice_padded.reshape(envelope_lattice.shape)

In [18]:
Quiteness_lattice =tg.to_lattice(Quiteness_lattice_np, Quiteness_lattice_np.shape)

# Find distances for Facade calculations

In [19]:
Y_coordinates= voxel_coordinates.T[1].flatten()

In [20]:
X_coordinates= voxel_coordinates.T[0].flatten()

In [21]:
Building_classification_north_south_facades =[]

for center in Y_coordinates:
    if center >= 18 and center <= 30:
        Building_classification_north_south_facades.append(center-17)
        #print("1st")
    elif center >= 40 and center <= 82:
        Building_classification_north_south_facades.append(center-41)
       # print("2nd")
    elif center >= 82:
        Building_classification_north_south_facades.append(center-83)
        #print("3rd")
    else:
        Building_classification_north_south_facades.append(center)
    

In [22]:
Building_classification_east_west_facades =[]

for center,value in zip(Y_coordinates,X_coordinates):
    if center <= 90:
        Building_classification_east_west_facades.append(10-value)
        #print("1st")
    elif center >= 91:
        Building_classification_east_west_facades.append(130-value)
        #print("2nd")

# Facade Distances to North Facade

In [23]:
Distance_inverse_y_axis_north = [i for i in reversed(Building_classification_north_south_facades)]

In [24]:
Distance_north_facade_full_lattice= np.tile(Distance_inverse_y_axis_north,352)

In [25]:
North_facade_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Distance_inverse_y_axis_north))])

In [26]:
North_facade_array = np.array(North_facade_lattice_padded)

In [27]:
north_facade_lattice_np = North_facade_array.reshape(envelope_lattice.shape)

In [28]:
North_facade_lattice =tg.to_lattice(north_facade_lattice_np, north_facade_lattice_np.shape)

# Facade Distances to South Facade

In [29]:
South_facade_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Building_classification_north_south_facades))])

In [30]:
Souh_facade_array = np.array(South_facade_lattice_padded)

In [31]:
South_facade_lattice_np = Souh_facade_array.reshape(envelope_lattice.shape)

In [32]:
South_facade_lattice =tg.to_lattice(South_facade_lattice_np, South_facade_lattice_np.shape)

# Facade Distances to West Facade

In [33]:
West_facade_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Building_classification_east_west_facades))])

In [34]:
West_facade_array = np.array(West_facade_lattice_padded)

In [35]:
West_facade_lattice_np = West_facade_array.reshape(envelope_lattice.shape)

In [36]:
West_facade_lattice =tg.to_lattice(West_facade_lattice_np, West_facade_lattice_np.shape)

# Facade Distances to East Facade

In [37]:
Distance_inverse_y_axis_west = [i for i in reversed(Building_classification_east_west_facades)]

In [38]:
West_facade_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Distance_inverse_y_axis_west))])

In [39]:
East_facade_array = np.array(West_facade_lattice_padded)

In [40]:
East_facade_lattice_np = East_facade_array.reshape(envelope_lattice.shape)

In [41]:
East_facade_lattice =tg.to_lattice(East_facade_lattice_np, East_facade_lattice_np.shape)

# Find distances from Ground calculations

In [42]:
all_heights= (np.arange(0,61,6))

In [43]:
Distance_from_ground_complete_lattice = np.tile(all_heights, int(envelope_lattice.size/11))

In [44]:
Distance_from_ground_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Distance_from_ground_complete_lattice))])

In [45]:
padded_ground_distance_array = np.array(Distance_from_ground_lattice_padded)

In [46]:
padded_ground_distance_array_np = padded_ground_distance_array.reshape(envelope_lattice.shape)

In [47]:
Distance_from_ground = tg.to_lattice(padded_ground_distance_array_np, padded_ground_distance_array_np.shape)

# Find distances from Roof calculations

In [48]:
all_heights_reversed= [i for i in reversed(all_heights)]

In [49]:
Distance_from_roof_complete_lattice = np.tile(all_heights_reversed, int(envelope_lattice.size/11))

In [50]:
Distance_from_roof_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Distance_from_roof_complete_lattice))])

In [51]:
padded_roof_distance_array = np.array(Distance_from_roof_lattice_padded)

In [52]:
padded_roof_distance_array_np =  padded_roof_distance_array.reshape(envelope_lattice.shape)

In [53]:
Distance_from_roof = tg.to_lattice(padded_roof_distance_array_np, padded_roof_distance_array_np.shape)

# Visualize the Quiteness Lattice

In [54]:
###### initiating the plotter
p = pv.Plotter(notebook=True)
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Quiteness"] = Quiteness_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume

opacity = [0, 0.75, 0, 0.75, 1.0] 
clim = [0, 100]
p.add_volume(grid, cmap="magma", clim=clim,
             opacity=opacity, opacity_unit_distance=5,)
p.add_points( Noise_sources, color='#e63946')
# plotting
#p.camera_position = [(87, 269, 373), (-300, 70, -0), (0, 0, 1)]
#p.image_depth
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
p.show(use_ipyvtk=True,screenshot='Quiteness_lattice.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

# Visualize the N Facade Lattice

In [55]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Distance from North Facade"] = North_facade_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume

opacity = [0, 0.75, 0.7, 0.75, 0.8]
clim = [0, 100]
p.add_volume(grid, cmap="viridis", opacity=opacity, shade=False)
# plotting
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
p.show(use_ipyvtk=True,screenshot='Distance_from_North_Facade.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

# Visualize the S Facade Lattice

In [56]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Distance from South Facade"] = South_facade_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume

opacity = [0, 0.75, 0.7, 0.75, 0.8]
clim = [0, 100]
p.add_volume(grid, cmap="viridis", opacity=opacity, shade=False)
# plotting
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
p.show(use_ipyvtk=True,screenshot='Distance_from_South_Facade.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

# Visualize the E Facade Lattice

In [57]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Distance from East Facade"] = East_facade_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
opacity = [0, 0.75, 0, 0.75, 1.0]
clim = [0, 100]
p.add_volume(grid, cmap="viridis", opacity=opacity, shade=False)
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
# plotting
p.show(use_ipyvtk=True,screenshot='Distance_from_East_Facade.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

# Visualize the W Facade Lattice

In [58]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Distance from West Facade"] = West_facade_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
opacity = [0, 0.75, 0, 0.75, 1.0]
clim = [0, 100]
p.add_volume(grid, cmap="viridis", opacity=opacity, shade=False)
# plotting
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
p.show(use_ipyvtk=True,screenshot='Distance_from_West_Facade.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

# Visualize the Terrace Lattice

In [59]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Distance from Roof"] = Distance_from_roof.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
opacity = [0, 0.75, 0, 0.75, 1.0]
clim = [0, 100]
p.add_volume(grid, cmap="viridis", opacity=opacity, shade=False)
# plotting
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
p.show(use_ipyvtk=True,screenshot='Distance_from_Roof.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

# Visualize the Ground lattice

In [69]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Distance from ground"] = Distance_from_ground.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume
opacity = [0, 0.75, 0, 0.75, 1.0] 
clim = [0, 100]
p.add_volume(grid, cmap="viridis", opacity=opacity, shade=False)
# plotting
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
p.show(use_ipyvtk=True,screenshot='Distance_from_Ground.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

In [61]:
North_facade_lattice

lattice([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0., 19., 19., 19., 19., 19., 19., 19., 19., 19.,  0.],
          [ 0., 13., 13., 13., 13., 13., 13., 13., 13., 13.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0., 37., 37., 37.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0., 31., 31., 31.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0., 25., 25., 25.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0., 19., 19., 19.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0., 13., 13., 13.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  7.,  7.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0., 36., 36., 36., 36., 36., 36., 36., 36., 36.,  0.],
          [ 0., 13., 13., 13., 13., 13., 13., 13., 13., 13.,  0.],
          [ 0.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  

# Pickle all Lattices

In [62]:
#Quiteness_mtrx = pickle.dump( Quiteness_lattice, open( "Quiteness_lattice.p", "wb" ) )

In [63]:
#North_facade_mtrx = pickle.dump( North_facade_lattice, open( "North_facade_lattice.p", "wb" ) )

In [64]:
#South_facade_mtrx = pickle.dump( South_facade_lattice, open( "South_facade_lattice.p", "wb" ) )

In [65]:
#East_facade_mtrx = pickle.dump( East_facade_lattice, open( "East_facade_lattice.p", "wb" ) )

In [66]:
#West_facade_mtrx = pickle.dump( West_facade_lattice, open( "West_facade_lattice.p", "wb" ) )

In [67]:
#Terrace_facade_mtrx = pickle.dump( Distance_from_roof, open( "Distance_from_roof.p", "wb" ) )

In [68]:
#Ground_facade_mtrx = pickle.dump( Distance_from_ground, open( "Distance_from_ground.p", "wb" ) )